----
<img src="../../files/refinitiv.svg" class="rft-examples-logo" width="20%" style="vertical-align: top;">

# Data Library for Python

----

## Content layer - Pricing stream - Used as a real-time data cache
This notebook demonstrates how to retrieve level 1 streaming data (such as trades and quotes) either directly from the Refinitiv Data Platform or via Refinitiv Workspace or CodeBook. The example shows how to define a Pricing stream object, which automatically manages a streaming cache available for access at any time. Your application can then reach into this cache and pull out real-time snapshots as Pandas DataFrames by just calling a simple access method.

Using a Pricing stream object that way prevents your application from sending too many requests to the platform. This is particularly useful if your application needs to retrieve real-time snapshots at regular and short intervals.

#### Learn more

To learn more about the Refinitiv Data Library for Python please join the Refinitiv Developer Community. By [registering](https://developers.refinitiv.com/iam/register) and [login](https://developers.refinitiv.com/content/devportal/en_us/initCookie.html) to the Refinitiv Developer Community portal you will get free access to a number of learning materials like 
 [Quick Start guides](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/quick-start), 
 [Tutorials](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/tutorials), 
 [Documentation](https://developers.refinitiv.com/en/api-catalog/refinitiv-data-platform/refinitiv-data-library-for-python/documentation)
 and much more.

#### Getting Help and Support

If you have any questions regarding the API usage, please post them on 
the [Refinitiv Data Q&A Forum](https://community.developers.refinitiv.com/spaces/321/index.html). 
The Refinitiv Developer Community will be happy to help.

----

## Some Imports to start with

In [1]:
import refinitiv.data as rd
from refinitiv.data.content import pricing
from pandas import DataFrame
from IPython.display import display, clear_output

## Open the data session

The open_session() function creates and open sessions based on the information contained in the refinitiv-data.config.json configuration file. Please edit this file to set the session type and other parameters required for the session you want to open.

In [2]:
rd.open_session()

<refinitiv.data.session.Definition object at 0x15f844370 {name='workspace'}>

## Retrieve data

### Create and open a Pricing stream object

The Pricing stream object is created for a list of instruments and fields. The fields parameter is optionnal. If you omit it, the Pricing stream will retrieve all fields available for the requested instruments

In [3]:
stream = rd.content.pricing.Definition(
    universe = ['EUR=', 'GBP=', 'JPY=', 'CAD='], 
    fields = ['BID', 'ASK']
).get_stream()

The open method tells the Pricing stream object to subscribe to the streams of the requested instruments.

In [4]:
stream.open()

<OpenState.Opened: 'Opened'>

As soon as the open method returns, the stream object is ready to be used. Its internal cache is constantly kept updated with the latest streaming information received from Eikon / Refinitiv Workspace. All this happens behind the scene, waiting for your application to pull out data from the cache.  

### Extract snapshot data from the streaming cache
Once the stream is opened, you can use the get_snapshot method to pull out data from its internal cache. get_snapshot can be called any number of times. As these calls return the latest received values, successive calls to get_snapshot may return different values. Returned DataFrames do not change in real-time, get_snapshot must be called every time your application needs fresh values. 

In [5]:
df = stream.get_snapshot()
display(df)

,Instrument,BID,ASK
0,EUR=,1.0684,1.0688
1,GBP=,1.2039,1.2043
2,JPY=,135.92,135.97
3,CAD=,1.361,1.3611


### Get a snapshot for a subset of instruments and fields

In [6]:
df = stream.get_snapshot(
    universe = ['EUR=', 'GBP='], 
    fields = ['BID', 'ASK']
)
display(df)

,Instrument,BID,ASK
0,EUR=,1.0683,1.0687
1,GBP=,1.2041,1.2042


### Other options to get values from the streaming cache

#### Direct access to real-time fields

In [7]:
print('GBP/BID:', stream['GBP=']['BID'])
print('EUR/BID:', stream['EUR=']['BID'])

GBP/BID: 1.2039
EUR/BID: 1.0683


#### Direct acces to a streaming instrument

In [8]:
gbp = stream['GBP=']
print(gbp['BID'])

1.2041


#### Iterate on fields

In [9]:
print('GBP=')
for field_name, field_value in stream['GBP=']:
    print('\t' + field_name + ': ', field_value)
    
print('JPY=')
for field_name, field_value in stream['JPY=']:
    print('\t' + field_name + ': ', field_value)

GBP=
	BID:  1.2041
	ASK:  1.2042
JPY=
	BID:  135.93
	ASK:  135.94


#### Iterate on streaming instruments and fields

In [10]:
for streaming_instrument in stream:
    print(streaming_instrument.name)
    for field_name, field_value in streaming_instrument:
        print('\t' + field_name + ': ', field_value)

EUR=
	BID:  1.0683
	ASK:  1.0687
GBP=
	BID:  1.2041
	ASK:  1.2042
JPY=
	BID:  135.92
	ASK:  135.95
CAD=
	BID:  1.3608
	ASK:  1.3612


### Close the stream

In [11]:
stream.close()

<OpenState.Closed: 'Closed'>

Once closed is called the Pricing stream object stops updating its internal cache. The get_snapshot function can still be called but after the close it always return the same values. 

### Invalid or un-licensed instruments
What happens if you request using an invalid RIC or an instrument you are not entitled to?
Let's request a mixture of valid and invalid RICs

In [12]:
mixed = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD=', 'BADRIC'],
    fields=['BID', 'ASK']
).get_stream()

mixed.open()
mixed.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.0683,1.0687
1,GBP=,1.204,1.2041
2,JPY=,135.92,135.95
3,CAD=,1.361,1.3611
4,BADRIC,<NA>,<NA>


You can check the Status of any instrument, so lets check the invalid one

In [13]:
display(mixed['BADRIC'].status)

{'ID': 13,
 'Type': 'Status',
 'Key': {'Service': 'IDN_RDFNTS_CF', 'Name': 'BADRIC'},
 'State': {'Stream': 'Closed',
  'Data': 'Suspect',
  'Code': 'NotFound',
  'Text': 'The record could not be found'}}

As you will note, for an invalid instrument we get:   
{'status': <StreamState.Closed: 1>, **'code': 'NotFound'**, 'message': '** The Record could not be found'}   

However, if you are not licensed for the instrument you would see something like:  
{'status': <StreamState.Closed: 1>, **'code': 'NotEntitled'**, 'message': 'A21: DACS User Profile denied access to vendor'}   
**NOTE**: The exact wording of **message** can change over time - therefore,only use the **code** value for any programmatic decision making.

In [14]:
mixed.close()

<OpenState.Closed: 'Closed'>

## Close the session

In [15]:
rd.close_session()